# Standard addition of SSDE without dependencies between the digits

In [1]:
load('common_computations-standard-addition.sage', 'common_computations-ssde.sage')
var('q')

q

First, we define the sets which are used to combine certain transitions:

In [2]:
digits_labels(q)

[-q,
 -H,
 -1/2*q - 1,
 -1/2*q,
 -1/2*q + 1,
 -L,
 0,
 L,
 1/2*q - 1,
 1/2*q,
 1/2*q + 1,
 H,
 q]

Now, we can construct the transducer to perform the addition:

In [3]:
def transformation(state, d, q):
    digit = digits(q)[d]
    if state == 'q/2':
        if digit == digits(q)[q]:
            new_carry = 1
            old_carry = 0
        elif greaterequal0(digit.min-q/2):
            new_carry = 1
            old_carry = 1
        elif greaterequal0(digit.min) and greater0(q/2-digit.max):
            new_carry = 0
            old_carry = 0
        elif greaterequal0(digit.min+q/2) and greater0(-digit.max):
            new_carry = 0
            old_carry = 1
        else:
            new_carry = -1
            old_carry = 0
        new_state = new_carry
        carry = [new_carry, old_carry]
    elif state == '-q/2':
        if digit == digits(q)[-q]:
            new_carry = -1
            old_carry = 0
        elif greaterequal0(-digit.max-q/2):
            new_carry = -1
            old_carry = -1
        elif greaterequal0(-digit.max) and greater0(digit.min+q/2):
            new_carry = 0
            old_carry = 0
        elif greaterequal0(-digit.max+q/2) and greater0(digit.min):
            new_carry = 0
            old_carry = -1
        else:
            new_carry = 1
            old_carry = 0
        new_state = new_carry
        carry = [new_carry, old_carry]
    else:
        current = digit + state
        if current == digits(q)[q/2]:
            carry = None
            return ('q/2', carry)
        elif current == digits(q)[-q/2]:
            carry = None
            return ('-q/2', carry)
        elif greater0(-q/2-current.max):
            carry = -1
        elif greater0(current.min-q/2):
            carry = 1
        else:
            carry = 0
        new_state = carry
    if carry is None:
        raise LookupError
    return (new_state, carry)
@cached_function
def Addition(q):
    def transformation_q(x, y):
        return transformation(x, y, q)
    return Transducer(transformation_q, 
    input_alphabet=digits_labels(q),
    initial_states=[0], final_states=[0])

Next, we construct the weighted automaton recognizing the digitwise sum of two SSDE:

In [4]:
DigitwiseSumFull = Transducer(initial_states=[(0, 0)], final_states=[(0, 0)],
    on_duplicate_transition=duplicate_transition_add_input)
def DigitwiseSumTransitionFunction((type_x_old, type_y_old), (type_x_new, type_y_new)):
    result = []
    for digit_label in digits_labels(q):
        digit = digits(q)[digit_label]
        probability = N(-q/2, -q/2, -q/2, -q/2, digit.min, digit.max) * (1/2/(q+1)) * (1/2/(q+1)) + \
                      N(-q/2, -q/2, -q/2+1, -1, digit.min, digit.max) * (1/2/(q+1)) * (1/q) * 2 + \
                      N(-q/2, -q/2, 0, 0, digit.min, digit.max) * (1/2/(q+1)) * ((q+2)/(q*(q+1))) * 2 + \
                      N(-q/2, -q/2, 1, q/2-1, digit.min, digit.max) * (1/2/(q+1)) * (1/q) * 2 + \
                      N(-q/2, -q/2, q/2, q/2, digit.min, digit.max) * (1/2/(q+1)) * (1/2/(q+1)) * 2 + \
                      N(-q/2+1, -1, -q/2+1, -1, digit.min, digit.max) * (1/q) * (1/q) + \
                      N(-q/2+1, -1, 0, 0, digit.min, digit.max) * (1/q) * ((q+2)/(q*(q+1))) * 2 + \
                      N(-q/2+1, -1, 1, q/2-1, digit.min, digit.max) * (1/q) * (1/q) * 2 + \
                      N(-q/2+1, -1, q/2, q/2, digit.min, digit.max) * (1/q) * (1/2/(q+1)) * 2 + \
                      N(0, 0, 0, 0, digit.min, digit.max) * ((q+2)/(q*(q+1))) * ((q+2)/(q*(q+1))) + \
                      N(0, 0, 1, q/2-1, digit.min, digit.max) * ((q+2)/(q*(q+1))) * (1/q) * 2 + \
                      N(0, 0, q/2, q/2, digit.min, digit.max) * ((q+2)/(q*(q+1))) * (1/2/(q+1)) * 2 + \
                      N(1, q/2-1, 1, q/2-1, digit.min, digit.max) * (1/q)*(1/q) + \
                      N(1, q/2-1, q/2, q/2, digit.min, digit.max) * (1/q)*(1/2/(q+1)) * 2 + \
                      N(q/2, q/2, q/2, q/2, digit.min, digit.max) * (1/2/(q+1)) * (1/2/(q+1))                        
        if probability != 0:
            result.append((probability.factor(), digit.symbol))
    return result
DigitwiseSumFull.add_transitions_from_function(DigitwiseSumTransitionFunction)
assert DigitwiseSumFull.is_Markov_chain()

In [5]:
DigitwiseSum = DigitwiseSumFull.markov_chain_simplification()
DigitwiseSum

Transducer with 1 state

In [6]:
qmin

8

Combining the these two finite state machine gives the weighted automaton S_SSDE:

In [7]:
sage.combinat.finite_state_machine.FSMOldProcessOutput = False
S_SSDE = (Addition(q))(DigitwiseSum)

Next, we construct the adjacency matrix and compute its determinant. For a better performance, we switch from the symbolic ring to a multivariate polynomial ring.

In [8]:
RQ.<y,q,x,z> = QQ[]
RQM = MatrixSpace(RQ, len(S_SSDE.states()))
def matrix_function(transition):
    return transition.word_in[0] * x^add(list(j for j in transition.word_out if j==1)) \
                * y^(-add(list(j for j in transition.word_out if j==-1)))
A = S_SSDE.adjacency_matrix(entry=matrix_function)
sysM = identity_matrix(ZZ,A.ncols()) - z*A
k=2
MQ = RQM(sysM*(q*(q+1))^k)
f = MQ.determinant() / (q*(q+1))^(k*A.ncols())
results = expectation_variance(f)
results

{'covariance': (1/64*q^38 + 1/2*q^37 + 411/64*q^36 + 1513/32*q^35 + 1835/8*q^34 + 25089/32*q^33 + 124465/64*q^32 + 111763/32*q^31 + 274257/64*q^30 + 42351/16*q^29 - 16953/8*q^28 - 275315/32*q^27 - 925415/64*q^26 - 38933/2*q^25 - 1744907/64*q^24 - 162729/4*q^23 - 3062951/64*q^22 - 204919/16*q^21 + 6169975/64*q^20 + 1925395/8*q^19 + 18578201/64*q^18 + 4479799/32*q^17 - 2141123/16*q^16 - 9572073/32*q^15 - 13221653/64*q^14 + 158003/4*q^13 + 3176427/16*q^12 + 1194123/8*q^11 - 28963/2*q^10 - 234591/2*q^9 - 344597/4*q^8 + 4839*q^7 + 49316*q^6 + 28176*q^5 - 4256*q^4 - 11088*q^3 - 3072*q^2 + 1152*q + 576)/(-q^38 - 20*q^37 - 187*q^36 - 1078*q^35 - 4237*q^34 - 11752*q^33 - 22481*q^32 - 25046*q^31 + 3133*q^30 + 75864*q^29 + 161153*q^28 + 170286*q^27 + 21812*q^26 - 245448*q^25 - 432928*q^24 - 319836*q^23 + 96254*q^22 + 503724*q^21 + 510196*q^20 + 52196*q^19 - 462041*q^18 - 531976*q^17 - 108801*q^16 + 349766*q^15 + 411992*q^14 + 128056*q^13 - 133000*q^12 - 159360*q^11 - 49440*q^10 + 26016*q^9 + 2739

In [9]:
(results['expectation_x'].factor(), results['expectation_y'].factor())

((1/8) * (q + 1)^-3 * q^-2 * (q^7 + 4*q^6 + 5*q^5 - q^4 - 9*q^3 - 8*q^2 + 4)^-1 * (q^12 + 7*q^11 + 19*q^10 + 27*q^9 + 24*q^8 + 9*q^7 - 15*q^6 - 15*q^5 + 47*q^4 + 104*q^3 + 64*q^2 - 48*q - 48),
 (1/8) * (q + 1)^-3 * q^-2 * (q^7 + 4*q^6 + 5*q^5 - q^4 - 9*q^3 - 8*q^2 + 4)^-1 * (q^12 + 7*q^11 + 19*q^10 + 27*q^9 + 24*q^8 + 9*q^7 - 15*q^6 - 15*q^5 + 47*q^4 + 104*q^3 + 64*q^2 - 48*q - 48))